In [ ]:
import petl as etl # для загрузки и обработки данных

from app_store_scraper import AppStore # для выгрузки отзывов о приложении
import pandas as pd # для удобной работы с датой и выгрузки таблицы в postgresql

import sqlalchemy # для создания таблицы в базе данных

In [ ]:
last_reviews = AppStore(country="ru", app_name="visiology mobile")
last_reviews.review(how_many=10)

In [ ]:
tbl = etl.fromdicts(last_reviews.reviews) # создание таблицы petl

In [ ]:
tbl = etl.cutout(tbl, 'isEdited', 'title') # удаление ненужных столбцов

In [ ]:
tbl.toxlsx('visiology_mobile_reviews.xlsx', write_header=True, mode='replace') # экспорт в Excel

In [ ]:
engine = sqlalchemy.create_engine('postgresql://{user}:{user_password}@{url}:{port}/{database_name}') # подключение к базе данных

In [ ]:
df = pd.DataFrame(columns=tbl[0], data=tbl[1:]) # создание DataFrame из petl-таблицы

In [ ]:
# экспорт в базу данных
df.to_sql('visiology_mobile_reviews', engine, index=False, if_exists='replace',
                    dtype={
                        'userName': sqlalchemy.VARCHAR(30),
                        'rating': sqlalchemy.Integer(),
                        'review': sqlalchemy.Text(),
                        'date' : sqlalchemy.DateTime(),
                        'title': sqlalchemy.VARCHAR(50)
                    })